# Processing 3D microscope images into 2D .tif files

In this jupyter notebook we will process 3D microscope images. We want to go from 3D .ims file generated by the Andor Dragonfly microscope to separate 2D .tif files for each channel, plus a scale bar image generated from the metadata of the .ims file. The resulting .tif files were combined into Adobe Photoshop using a separate script (also provided). 

This code performs several steps: 
- Autoscales the minimum and maximum signals to save time in Photoshop later. 
- Turns the 3D into 2D by maximum intensity projection (MIP). 
- Parses the 4 channel image into separate .tif files per channel. 
- Generates a scalebar .tif file based on metadata inside the .ims file. 
- 4 channel .tif files and scalebar .tif files are deposited into a separate folder. 


Of note: 
- In your directory where this .ipynb notebook is stored, please make a folder and name this "Images" -> put your .ims files in this "Images" folder. 
- In your directory where this .ipynb notebook is stored, please make a folder and name this "Processed"
- The code assumes 16-bit images. 
- The code cannot handle video data (i.e., datasets with multiple timepoints). 

To start, we import the dependencies:
- some packages part of the standard python library, like pathlib;
- h5py is needed for opening ims files, which use the hdf5 standard;
- numpy is needed for the processing of the data.

In [1]:
from pathlib import Path
import json
import numpy as np
import h5py
import tifffile as tiff

First, we define a few functions so that our code later on is cleaner:

In [2]:
def get_abbreviated_name(name: str) -> str:
    """
    Returns an abbreviated name: creates a string of the first two characters of every word.
    """
    return "".join([part[:2] for part in name.capitalize().split(" ")])

def rescale_data(data, desired_min=0, desired_max=2**16):
    """
    Rescales numpy array (data) to be between desired_min and desired_max (linearly).
    Defaults to 16 bit rescale.
    """
    norm_data = (data - np.min(data)) / (np.max(data) - np.min(data))
    return (norm_data * (desired_max - desired_min)) + desired_min



Next is the code that generates the output .tif files from your input .ims files. 

You will need to input your .ims filename in the code. In addition, the current scalebar length is for 50 um, but you may adjust this length if you wish. 

In [15]:
ROOT_FOLDER = Path("__file__").parent

# Make sure your '.ims' files are in this folder:
SOURCE_FOLDER = ROOT_FOLDER / "Images"

# Make sure you created a 'Processed' folder:
PROCESSED_FOLDER = ROOT_FOLDER / "Processed"

# Edit these if you want to change scale bar length, height or position:
SCALE_BAR_LENGTH_MICROMETER = 50
SCALE_BAR_HEIGHT_PIXEL = 9
SCALE_BAR_RIGHT_BOTTOM_POSITION = (50, 50)


#############################################

# Here, type the filename of your .ims file (DO NOT INCLUDE '.ims'):
fname = "YOUR_IMAGE_FILE_NAME"

#############################################


f = h5py.File(SOURCE_FOLDER / f"{fname}.ims", 'r')

dataset = f.get("DataSet")
if not dataset:
    raise ValueError("Dataset not found in image")

resolutions = list(dataset.keys())

resolution = resolutions[0]
resolution_dataset = dataset[resolution]
timepoints = list(resolution_dataset.keys())
if len(timepoints) > 1:
    raise ValueError("This script does not know how to deal with datasets containing multiple timepoints. " \
          "Please create an issue on github and attach the image throwing this error.")


channels = resolution_dataset[timepoints[0]].items()

image_data_dict = {}
for channel_name, channel_data in channels:
    data = channel_data["Data"]
    data_array = np.zeros(shape=data.shape, dtype=data.dtype)
    data.read_direct(data_array)

    # Get maximum signal of all layers so we end up with one image
    reduced_image = np.maximum.reduce(data_array)

    # Rescale image to be between 0 and 2**16
    rescaled_image = rescale_data(reduced_image).astype("uint16")

    # Save to dictionary
    image_data_dict[get_abbreviated_name(channel_name)] = rescaled_image

    # Save to file
    output_file = ROOT_FOLDER / "Processed" / f"{fname}{get_abbreviated_name(resolution)}__{get_abbreviated_name(channel_name)}.tif"
    tiff.imwrite(output_file, rescaled_image)

# Scale bar
metadata = f["DataSetInfo"]["CustomData"]["PSF Settings Configuration V2"]
metadata_arr = np.zeros(metadata.shape, dtype=metadata.dtype)
metadata.read_direct(metadata_arr)

metadata_dict = json.loads(metadata_arr.tobytes().decode("iso-8859-1"))

resolution_x, resolution_y = metadata_dict["ResolutionX"], metadata_dict["ResolutionY"]
assert resolution_x == resolution_y

scale_bar_length_pixels = SCALE_BAR_LENGTH_MICROMETER / resolution_x

# Mirror image shape and dtype of rescaled_image
scale_bar_image = np.zeros(shape=rescaled_image.shape, dtype=rescaled_image.dtype)
scale_bar = np.ones((SCALE_BAR_HEIGHT_PIXEL, round(scale_bar_length_pixels)))
scale_bar_image[-scale_bar.shape[0]-SCALE_BAR_RIGHT_BOTTOM_POSITION[0]:-SCALE_BAR_RIGHT_BOTTOM_POSITION[0], -scale_bar.shape[1]-SCALE_BAR_RIGHT_BOTTOM_POSITION[1]:-SCALE_BAR_RIGHT_BOTTOM_POSITION[1]] = (2**16-1)*scale_bar

output_file = PROCESSED_FOLDER / f"{fname}{get_abbreviated_name(resolution)}__scale_bar_{SCALE_BAR_LENGTH_MICROMETER}micrometer.tif"
tiff.imwrite(output_file, scale_bar_image)

f.close()

### Below are some lines to call information from the image metadata. 

In [ ]:
metadata_dict

In [ ]:
[f["DataSet"]['ResolutionLevel 0']['TimePoint 0'].keys()]

In [ ]:
for e in arr:
    print(e.encode("utf-8"))

In [ ]:
print("".join(str(e) for e in arr).replace("b'", "").replace("'", ""))

In [12]:
tiff_file = TiffFile(ROOT_FOLDER / "example_tiff.tif")

In [72]:
tiff_file.close()

In [ ]:
for page in tiff_file.pages:
    print(page)

In [ ]:
f["DataSetInfo"].keys()

In [ ]:
f["DataSetInfo"]["CustomData"].keys()

In [ ]:
f["DataSetInfo"]["CustomData"]["Protocol Configuration V2"]

In [20]:
info = f["DataSetInfo"]["CustomData"]["Protocol Configuration V2"]
arr = np.zeros(info.shape, dtype=info.dtype)
info.read_direct(arr)

In [ ]:

import json
json.loads(arr.tobytes().decode("iso-8859-1"))

In [ ]:
f["DataSetInfo"]["CustomData"].keys()

In [ ]:
f["DataSetInfo"]["CustomData"]["Protocol Configuration V2"]

In [83]:
rescaled_image.shape

(1024, 1024)

In [ ]:
metadata_dict

In [ ]:
tif_tags.get("ImageDescription")